## CCTV 개수와 서울시 예산 비교하기

### 현재 당면한 과제

* Google Maps API를 활용한 심층적 시각화 (단, "CCTV와 범죄", "예산과 범죄"의 관점에 한하여)
* CCTV 과밀 지역을 판별하고, 해당 지역에 배정된 예산 및 해당 지역의 세입을 조사한다 
* CCTV 및 인구 데이터를 2024년 것으로 맞추어 업데이트


In [4]:
import pandas as pd
import numpy as np
import platform

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unable to acknowldege system. Maybe Linux?')

In [5]:
SeoulCCTV = pd.read_excel('../data/Seoul_Crime_CCTV_240630.xlsx')

# drop first row and first column from SeoulCCTV
SeoulCCTV.drop([0], inplace=True)
SeoulCCTV.drop(['Unnamed: 0'], axis=1, inplace=True)
SeoulCCTV.drop(['Unnamed: 1'], axis=1, inplace=True)

# use first row as column names
SeoulCCTV.columns = SeoulCCTV.iloc[0]

# drop first row and last row
SeoulCCTV.drop([1], inplace=True)
SeoulCCTV.drop([28], inplace=True)

# rename "구분" to "구별"
SeoulCCTV.rename(columns={SeoulCCTV.columns[0]:'구별'}, inplace=True)

# rename the value in second row, first column to "소계"
SeoulCCTV.iloc[0, 0] = '소계'
SeoulCCTV.head()

1,구별,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년,2023년,2024년 6.30.
2,소계,26321,33013,40512,49222,58139,67281,74408,80005,86810,89459
3,종로구,935,1066,1225,1322,1327,1510,1573,1812,1872,2154
4,중구,363,565,838,1174,1242,1482,1911,2026,2157,2460
5,용산구,1398,1689,1831,1888,1915,2058,2321,2531,2897,2938
6,성동구,1089,1328,2103,2390,2833,3162,3519,3627,3871,3971


In [6]:
SeoulPops = pd.read_excel('../data/Seoul_Pops_240807.xlsx',
                          header = 2,
                          usecols = 'B, D, G, J, N')
                          
SeoulPops.rename(columns={SeoulPops.columns[0] : '구별', 
                          SeoulPops.columns[1] : '인구수', 
                          SeoulPops.columns[2] : '한국인', 
                          SeoulPops.columns[3] : '외국인', 
                          SeoulPops.columns[4] : '고령자'}, inplace=True)

SeoulPops.head()


c:\Cookiebake\envs\Walter\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,구별,인구수,한국인,외국인,고령자
0,소계,9619861,9366283,253578,1785286
1,종로구,150204,139189,11015,29338
2,중구,131800,121475,10325,26516
3,용산구,219451,206061,13390,39021
4,성동구,283853,276597,7256,50534


In [7]:
SeoulPops['외국인비율'] = SeoulPops['외국인'] / SeoulPops['인구수'] * 100
SeoulPops['고령자비율'] = SeoulPops['고령자'] / SeoulPops['인구수'] * 100
SeoulPops.head()

,구별,인구수,한국인,외국인,고령자,외국인비율,고령자비율
0,소계,9619861,9366283,253578,1785286,2.635984,18.558335
1,종로구,150204,139189,11015,29338,7.333360,19.532103
2,중구,131800,121475,10325,26516,7.833839,20.118361
3,용산구,219451,206061,13390,39021,6.101590,17.781190
4,성동구,283853,276597,7256,50534,2.556253,17.802877


In [8]:
# SeoulBudget = pd.read_csv('../data/00. Seoul_Annual_Budget.csv', header = 2, usecols = ['B', 'C', 'F', 'I'], encoding='utf-8')
# read Seoul_Annual_Budget.xlsx file
SeoulBudget = pd.read_excel('../data/Seoul_Annual_Budget.xlsx', header = 2, usecols = 'B, C, F, I')

SeoulBudget.columns = ['구별', '총예산', '세출', '세입']
SeoulBudget.drop([0], inplace=True)

SeoulBudget.head()

c:\Cookiebake\envs\Walter\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,구별,총예산,세출,세입
1,소계,80797281,84378232,73233977
2,본청,53468799,55571683,50276587
3,자치구,27328481,28806549,22957390
4,종로구,746231,766207,613919
5,중구,710863,710895,562557


In [9]:
print(len(SeoulCCTV['구별'].unique()))
print(len(SeoulPops['구별'].unique()))
print(SeoulCCTV['구별'].unique())

26
26
['소계' '종로구' '중구' '용산구' '성동구' '광진구' '동대문구' '중랑구' '성북구' '강북구' '도봉구' '노원구'
 '은평구' '서대문구' '마포구' '양천구' '강서구' '구로구' '금천구' '영등포구' '동작구' '관악구' '서초구' '강남구'
 '송파구' '강동구']


In [10]:
if(len(SeoulCCTV['구별'].unique()) == len(SeoulPops['구별'].unique())):
    print('구별 컬럼이 일치합니다.')
else:
    print('구별 컬럼이 일치하지 않습니다.')
    
SeoulPrep = pd.merge(SeoulCCTV, SeoulPops, on='구별')
SeoulPrep.head()

구별 컬럼이 일치합니다.


,구별,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년,2023년,2024년 6.30.,인구수,한국인,외국인,고령자,외국인비율,고령자비율
0,소계,26321,33013,40512,49222,58139,67281,74408,80005,86810,89459,9619861,9366283,253578,1785286,2.635984,18.558335
1,종로구,935,1066,1225,1322,1327,1510,1573,1812,1872,2154,150204,139189,11015,29338,7.333360,19.532103
2,중구,363,565,838,1174,1242,1482,1911,2026,2157,2460,131800,121475,10325,26516,7.833839,20.118361
3,용산구,1398,1689,1831,1888,1915,2058,2321,2531,2897,2938,219451,206061,13390,39021,6.101590,17.781190
4,성동구,1089,1328,2103,2390,2833,3162,3519,3627,3871,3971,283853,276597,7256,50534,2.556253,17.802877


In [11]:
# Don't run this cell unless you want to refine the data

# drop columns from the second to the tenth 
SeoulPrep.drop(SeoulPrep.columns[1:10], axis=1, inplace=True)

# drop first row
SeoulPrep.drop([0], inplace=True)

In [12]:
SeoulPrep.head()

,구별,2024년 6.30.,인구수,한국인,외국인,고령자,외국인비율,고령자비율
1,종로구,2154,150204,139189,11015,29338,7.333360,19.532103
2,중구,2460,131800,121475,10325,26516,7.833839,20.118361
3,용산구,2938,219451,206061,13390,39021,6.101590,17.781190
4,성동구,3971,283853,276597,7256,50534,2.556253,17.802877
5,광진구,3722,350005,334159,15846,58307,4.527364,16.658905


In [13]:
SeoulPrep.rename(columns={SeoulPrep.columns[1] : 'CCTV'}, inplace=True)
SeoulPrep.head()

,구별,CCTV,인구수,한국인,외국인,고령자,외국인비율,고령자비율
1,종로구,2154,150204,139189,11015,29338,7.333360,19.532103
2,중구,2460,131800,121475,10325,26516,7.833839,20.118361
3,용산구,2938,219451,206061,13390,39021,6.101590,17.781190
4,성동구,3971,283853,276597,7256,50534,2.556253,17.802877
5,광진구,3722,350005,334159,15846,58307,4.527364,16.658905


In [14]:
import googlemaps

gmaps_key = "NotKey"
gmaps = googlemaps.Client(key=gmaps_key)

In [15]:
gmaps.geocode("서울시청", language="ko")

[{'address_components': [{'long_name': '110',
    'short_name': '110',
    'types': ['premise']},
   {'long_name': '세종대로',
    'short_name': '세종대로',
    'types': ['political', 'sublocality', 'sublocality_level_4']},
   {'long_name': '중구',
    'short_name': '중구',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': '서울특별시',
    'short_name': '서울특별시',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': '대한민국',
    'short_name': 'KR',
    'types': ['country', 'political']},
   {'long_name': '04524', 'short_name': '04524', 'types': ['postal_code']}],
  'formatted_address': '대한민국 서울특별시 중구 세종대로 110',
  'geometry': {'location': {'lat': 37.5665851, 'lng': 126.9782038},
   'location_type': 'ROOFTOP',
   'viewport': {'northeast': {'lat': 37.56793408029149,
     'lng': 126.9795527802915},
    'southwest': {'lat': 37.5652361197085, 'lng': 126.9768548197085}}},
  'partial_match': True,
  'place_id': 'ChIJKwjLMvOifDURqPAMQqxwK-k',
  'plus_code'

In [16]:
center_name = []

for name in SeoulPrep["구별"]:
    center_name.append(name + "청")
    
center_name

['종로구청',
 '중구청',
 '용산구청',
 '성동구청',
 '광진구청',
 '동대문구청',
 '중랑구청',
 '성북구청',
 '강북구청',
 '도봉구청',
 '노원구청',
 '은평구청',
 '서대문구청',
 '마포구청',
 '양천구청',
 '강서구청',
 '구로구청',
 '금천구청',
 '영등포구청',
 '동작구청',
 '관악구청',
 '서초구청',
 '강남구청',
 '송파구청',
 '강동구청']

In [17]:
center_address = []
center_lat = []
center_lng = []
temp = [] 

for name in center_name:
    temp = gmaps.geocode(name, language="ko")
    if temp:
        center_address.append(temp[0].get("formatted_address"))
        
        temp_loc = temp[0].get("geometry")
        center_lat.append(temp_loc['location']['lat'])
        center_lng.append(temp_loc['location']['lng'])
        
        print(name + '-->' + temp[0].get("formatted_address"))
    elif not temp:
        # if there is no result, check "서울특별시" + name
        temp = gmaps.geocode("서울특별시 " + name, language="ko")
        center_address.append(temp[0].get("formatted_address"))
        
        temp_loc = temp[0].get("geometry")
        center_lat.append(temp_loc['location']['lat'])
        center_lng.append(temp_loc['location']['lng'])
        
        print(name + '-->' + temp[0].get("formatted_address"))
    else:
        print("No Result for " + name)
    

종로구청-->대한민국 서울특별시 종로구 종로1길 36
중구청-->대한민국 서울특별시 중구 창경궁로 17
용산구청-->대한민국 서울특별시 용산구 녹사평대로 150
성동구청-->대한민국 서울특별시 성동구 고산자로 270
광진구청-->대한민국 서울특별시 광진구 자양로 117
동대문구청-->대한민국 서울특별시 동대문구 천호대로 145
중랑구청-->대한민국 서울특별시 중랑구 봉화산로 179
성북구청-->대한민국 서울특별시 성북구 보문로 168
강북구청-->대한민국 서울특별시 강북구
도봉구청-->대한민국 서울특별시 도봉구 마들로 656
노원구청-->대한민국 서울특별시 노원구 노해로 437
은평구청-->대한민국 서울특별시 은평구 은평로 195
서대문구청-->대한민국 서울특별시 서대문구 연희로 248
마포구청-->대한민국 서울특별시 마포구
양천구청-->대한민국 서울특별시 양천구 목동동로 105
강서구청-->대한민국 서울특별시 강서구 화곡로 302
구로구청-->대한민국 서울특별시 구로구 가마산로 245
금천구청-->대한민국 서울특별시 금천구
영등포구청-->대한민국 서울특별시 영등포구 당산로 123
동작구청-->대한민국 서울특별시 동작구
관악구청-->대한민국 서울특별시 관악구 관악로 145
서초구청-->대한민국 서울특별시 서초구 남부순환로 2584
강남구청-->대한민국 서울특별시 강남구
송파구청-->대한민국 서울특별시 송파구
강동구청-->대한민국 서울특별시 강동구 성내로 25


In [18]:
center_address

['대한민국 서울특별시 종로구 종로1길 36',
 '대한민국 서울특별시 중구 창경궁로 17',
 '대한민국 서울특별시 용산구 녹사평대로 150',
 '대한민국 서울특별시 성동구 고산자로 270',
 '대한민국 서울특별시 광진구 자양로 117',
 '대한민국 서울특별시 동대문구 천호대로 145',
 '대한민국 서울특별시 중랑구 봉화산로 179',
 '대한민국 서울특별시 성북구 보문로 168',
 '대한민국 서울특별시 강북구',
 '대한민국 서울특별시 도봉구 마들로 656',
 '대한민국 서울특별시 노원구 노해로 437',
 '대한민국 서울특별시 은평구 은평로 195',
 '대한민국 서울특별시 서대문구 연희로 248',
 '대한민국 서울특별시 마포구',
 '대한민국 서울특별시 양천구 목동동로 105',
 '대한민국 서울특별시 강서구 화곡로 302',
 '대한민국 서울특별시 구로구 가마산로 245',
 '대한민국 서울특별시 금천구',
 '대한민국 서울특별시 영등포구 당산로 123',
 '대한민국 서울특별시 동작구',
 '대한민국 서울특별시 관악구 관악로 145',
 '대한민국 서울특별시 서초구 남부순환로 2584',
 '대한민국 서울특별시 강남구',
 '대한민국 서울특별시 송파구',
 '대한민국 서울특별시 강동구 성내로 25']

In [19]:
center_lat

[37.5734684,
 37.5637584,
 37.5323264,
 37.5634092,
 37.53837420000001,
 37.5742015,
 37.60630460000001,
 37.589366,
 37.6444906,
 37.6687735,
 37.6540782,
 37.602749,
 37.5792607,
 37.5580889,
 37.5169508,
 37.5509103,
 37.4954703,
 37.4605655,
 37.52626250000001,
 37.4988794,
 37.4782605,
 37.4835872,
 37.4966645,
 37.5056205,
 37.530122]

In [20]:
center_lng

[126.978984,
 126.9975517,
 126.9907031,
 127.0369449,
 127.0822077,
 127.0398327,
 127.0931523,
 127.016743,
 127.010823,
 127.047071,
 127.0566045,
 126.929256,
 126.9364946,
 126.9083451,
 126.8665644,
 126.8495742,
 126.8876391,
 126.9008183,
 126.8959528,
 126.9516345,
 126.9515208,
 127.0326987,
 127.0629804,
 127.1152992,
 127.1237479]

In [21]:
SeoulBudget.sort_values(by='구별', ascending=True, inplace=True)
SeoulBudget.head()

,구별,총예산,세출,세입
26,강남구,1499273,1652978,1250825
28,강동구,1237844,1289018,1049127
12,강북구,1059703,1127810,910812
19,강서구,1439528,1476278,1277854
24,관악구,1254073,1307239,1097285


In [22]:
SeoulData = pd.merge(SeoulPrep, SeoulBudget, on='구별')
del SeoulData['세출']

In [23]:
SeoulData.drop(SeoulData.columns[3:6], axis=1, inplace=True)
SeoulData.head()

,구별,CCTV,인구수,외국인비율,고령자비율,총예산,세입
0,종로구,2154,150204,7.333360,19.532103,746231,613919
1,중구,2460,131800,7.833839,20.118361,710863,562557
2,용산구,2938,219451,6.101590,17.781190,744324,600039
3,성동구,3971,283853,2.556253,17.802877,899755,772494
4,광진구,3722,350005,4.527364,16.658905,987742,812094


In [24]:
# rename row '총예산' to '예산(만원)'
SeoulData.rename(columns={SeoulData.columns[5] : '예산(만원)'}, inplace=True)
SeoulData.head()

,구별,CCTV,인구수,외국인비율,고령자비율,예산(만원),세입
0,종로구,2154,150204,7.333360,19.532103,746231,613919
1,중구,2460,131800,7.833839,20.118361,710863,562557
2,용산구,2938,219451,6.101590,17.781190,744324,600039
3,성동구,3971,283853,2.556253,17.802877,899755,772494
4,광진구,3722,350005,4.527364,16.658905,987742,812094


In [25]:
SeoulData.sort_values(by='CCTV', ascending=False, inplace=True)
SeoulData.head()

,구별,CCTV,인구수,외국인비율,고령자비율,예산(만원),세입
22,강남구,6829,562136,0.990152,15.960373,1499273,1250825
16,구로구,4675,413099,5.550728,19.843427,1154912,950694
11,은평구,4369,468976,0.903671,20.507872,1313318,1107899
20,관악구,4269,497632,3.313091,17.536453,1254073,1097285
7,성북구,4181,436666,2.856188,18.770639,1234720,1039289


In [27]:
import folium
from folium import Choropleth

import json

geo_path = '../data/02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))


In [28]:
geo_names = [feature['properties']['name'] for feature in geo_str['features']]
a = len(geo_names)
b = len(SeoulData['구별'].unique())

print(a, b)

if a == b:
    print("this is good")
else:   
    print("this is not good")
    
print(geo_names)
print(SeoulData['구별'].unique())

geo_names_new = []
geo_names_new = SeoulData['구별'].unique()


25 25
this is good
['강동구', '송파구', '강남구', '서초구', '관악구', '동작구', '영등포구', '금천구', '구로구', '강서구', '양천구', '마포구', '서대문구', '은평구', '노원구', '도봉구', '강북구', '성북구', '중랑구', '동대문구', '광진구', '성동구', '용산구', '중구', '종로구']
['강남구' '구로구' '은평구' '관악구' '성북구' '중랑구' '영등포구' '양천구' '성동구' '광진구' '송파구' '강서구'
 '강동구' '강북구' '서초구' '마포구' '동대문구' '서대문구' '금천구' '용산구' '동작구' '노원구' '중구' '도봉구'
 '종로구']


In [47]:
# make a map via folium with SeoulData, focus on number of CCTV
maps = []

# Create the first map with locked zoom
map1 = folium.Map(
    location=[37.5502, 126.982], 
    zoom_start=10, 
    tiles='OpenStreetMap', 
    attr='Jinseong Choi Testing, 20240812',
    scrollWheelZoom=False,
    zoomControl=False
)
Choropleth(
    geo_data=geo_str,
    data=SeoulData,
    columns=['구별', 'CCTV'],
    fill_color='Blues',
    key_on='feature.properties.name',
    legend_name='CCTV'
).add_to(map1)

maps.append(map1)
map1

In [48]:
# Create the second map with locked zoom
map2 = folium.Map(
    location=[37.5502, 126.982], 
    zoom_start=10, 
    tiles='OpenStreetMap', 
    attr='Jinseong Choi Testing, 20240812',
    scrollWheelZoom=False,
    zoomControl=False
)
Choropleth(
    geo_data=geo_str,
    data=SeoulData,
    columns=['구별', '예산(만원)'],
    fill_color='PuRd',
    key_on='feature.properties.name',
    legend_name='예산(만원)'
).add_to(map2)

maps.append(map2)
map2

In [49]:
# 두 개의 지도를 오버랩하여 겹치는 장소에 대해 확인하기 
import folium
from folium import Choropleth
from IPython.display import display, HTML

# Create HTML to display maps side by side
html_str = f"""
<div style="display: flex;">
    <div style="flex: 1;">{map1._repr_html_()}</div>
    <div style="flex: 1;">{map2._repr_html_()}</div>
</div>
"""

# Display the HTML
display(HTML(html_str))

In [34]:
maps = []

mapCol = {('구별', 'CCTV'), ('구별', '예산(만원)')}
mapCol = list(mapCol)

# use for loop to create map twice with mapCol
for i in range(2):
    maps.append(folium.Map(location=[37.5502, 126.982], zoom_start=10, tiles='OpenStreetMap', attr='Jinseong Choi Testing, 20240812'))
    Choropleth(
        geo_data = geo_str,
        data = SeoulData,
        columns = mapCol[i],
        fill_color = 'Blues',
        key_on = 'feature.properties.name',
        legend_name='CCTV'
    ).add_to(maps[i])
    
# display maps[0] and maps[1] in a single screen
maps[0], maps[1]

(<folium.folium.Map at 0x219d83e79a0>, <folium.folium.Map at 0x219d8407a60>)